# Modularity

During the development process, we may have already had some individual automas. Now, we want to combine them to create a more powerful automa. **An automa can also be added as a Worker to another Automa to achieve reuse**.

Let's understand it with a sample case.

## Report Writer

The typical process of report writing usually involves drafting an outline based on user input and then writing the report according to the outline. Now, let's first write an Automa for generating an outline, and then nest it within the Automa for writing the report.

### 1. Initialize

Initialize the runtime environment.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

# Set the API base and key.
_api_base = os.environ.get("VLLM_SERVER_API_BASE")
_api_key = os.environ.get("VLLM_SERVER_API_KEY")
_model_name = os.environ.get("VLLM_SERVER_MODEL_NAME")

# Import the necessary modules.
from typing import List
from pydantic import BaseModel, Field
from bridgic.core.automa import GraphAutoma, worker
from bridgic.core.model.types import Message, Role
from bridgic.core.model.protocols import PydanticModel
from bridgic.llms.vllm import VllmServerLlm

llm = VllmServerLlm(api_base=_api_base, api_key=_api_key, timeout=30)

### 2. Outline Automa

Outline Automa drafts an outline based on user input. Output an `Outline` object for subsequent processing.

In [ ]:
class Outline(BaseModel):
    outline: List[str] = Field(description="The outline of the use input to write report")

class OutlineWriter(GraphAutoma):
    @worker(is_start=True)
    async def pre_query(self, user_input: str):  # Receive the user's input and preprocess query
        return user_input

    @worker(dependencies=["pre_query"], is_output=True)
    async def topic_split(self, query: str):
        response: Outline = await llm.astructured_output(
            model=_model_name,
            messages=[
                Message.from_text(text="Write a sample report outline within 3 topics based on the user's input", role=Role.SYSTEM),
                Message.from_text(text=query, role=Role.USER,),
            ],
            constraint=PydanticModel(model=Outline)
        )
        return response

### 3. Write Automa

We can reuse the `OutlineWriter` in `ReportWriter`.

In [ ]:
class ReportWriter(GraphAutoma):
    @worker(dependencies=["outline_writer"])
    async def write_report(self, outline: Outline):
        outline_str = "\n".join(outline.outline)
        print(f'- - - - - Outline - - - - -')
        print(outline_str)
        print(f'- - - - - End - - - - -\n')
        
        response = await llm.achat(
            model=_model_name,
            messages=[
                Message.from_text(text="write a sample report based on the user's input and strictly follow the outline", role=Role.SYSTEM),
                Message.from_text(text=f"{outline_str}.", role=Role.USER),
            ],
        )
        print(f'- - - - - Report - - - - -')
        print(response.message.content)
        print(f'- - - - - End - - - - -\n')
        return response.message.content

report_writer = ReportWriter()
report_writer.add_worker(
    key="outline_writer",
    worker=OutlineWriter(),
    is_start=True
)

Now, let's run it!

In [19]:
await report_writer.arun(user_input="What is an agent?")

- - - - - Outline - - - - -
Definition of an Agent
Types of Agents in Different Contexts
Role and Function of Agents in AI and Automation
- - - - - End - - - - -

- - - - - Report - - - - -
**Report: Definition of an Agent, Types of Agents in Different Contexts, and Role and Function of Agents in AI and Automation**

---

**1. Definition of an Agent**

An *agent* is an autonomous entity that perceives its environment through sensors and acts upon that environment using actuators to achieve specific goals. In a broad sense, an agent is capable of making decisions based on its internal state and external inputs, adapting its behavior over time in response to changing conditions. The concept of an agent is foundational in artificial intelligence (AI), robotics, and automated systems. It operates independently or in coordination with other agents, following predefined rules, learning from experience, or using reasoning mechanisms to perform tasks efficiently.

In computational terms, an ag

Great! We successfully reused `OutlineWriter` in `ReportWriter`. 

## What have we done

we added an automa as a `Worker` to another `Automa` to achieve nested reuse. There are several points that need special attention during the reuse process:

1. **Worker Dependency**: A `Worker` in an `Automa` can only depend on other workers within the same `Automa`, and cannot depend across different `Automa`.
2. **Routing**: A `Worker` in an `Automa` can only route to other workers within the same `Automa`, and cannot route across different `Automa`.
3. **Human-in-the-loop**: When a worker inside throws an event, it hands over this event to the automa agent for handling. At this point, the event handling functions of nested automa need to be registered to the outermost Automa.